In [1]:
%matplotlib inline
import os
import ipywidgets as widgets
from matplotlib import pyplot as plt

file_path = widgets.Text(
    description = 'path'
)

display(file_path)

Text(value='', description='path')

In [76]:
import configparser
config = configparser.ConfigParser()
config.read(file_path.value)

im_size = int(config['Training']['im_size'])
raw_path = config['Training']['raw_path']
ti_path = config['Training']['ti_path']
label_path = config['Training']['label_path']

In [77]:
import data_preprocessor as dp
import numpy as np
import skimage as ski

img = ski.io.imread(raw_path)

left_over = np.array(img.shape) % im_size
if(left_over[0] != 0):
    img = img[0:-left_over[0],:,:]
if(left_over[1] != 0):
    img = img[:,0:-left_over[1],:]
if(left_over[2] != 0):
    img = img[:,:,0:-left_over[2]]

train_set = dp.cubify(img, (im_size,im_size,im_size))


In [78]:
from ipywidgets import interactive

def f(img):
    def func(i,z):
        fig, ax = plt.subplots()
        ax.imshow(img[i,:,:,z],cmap = 'gray')
        plt.show()
    return func

disp_img = f(train_set)

interactive_plot = interactive(disp_img, i=(0, train_set.shape[0]), z=(0,train_set.shape[3]))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=108, description='i', max=216), IntSlider(value=32, description='z', max…

In [79]:
down_scaled = []

for img in train_set:
    x_center = img.shape[0] // 2
    y_center = img.shape[1] // 2
    z_center = img.shape[2] // 2
    
    img[x_center-20 : x_center+20, y_center-20 : y_center+20, z_center-20 : z_center+20] = 0
    inv_img = np.fft.ifftn(img)
    
    down_sp = ski.transform.downscale_local_mean(abs(inv_img),(7,7,7))
    #noisy_down_sp = ski.util.random_noise(down_sp)
    
    down_scaled.append(down_sp)
    

In [80]:
ti_data = np.array(down_scaled)

In [81]:
disp_img = f(ti_data)

interactive_plot = interactive(disp_img, i=(0, ti_data.shape[0]), z=(0,ti_data.shape[3]))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=108, description='i', max=216), IntSlider(value=5, description='z', max=…

In [83]:
import h5py

hf = h5py.File(label_path, "w")
data_set = hf.create_dataset("data", data=train_set.astype(np.float64))
hf.close()

hf = h5py.File(ti_path, "w")
data_set = hf.create_dataset("data", data=ti_data.astype(np.float64))
hf.close()
